Need experimental calculation:
$$\triangle_2 f_i = \left(\text{max}_{p,q} \frac{\left \| d_p^{(i)} - d_q^{(i)} \right \|_2}{n_i}\right)$$
$d$ is data embedding, $n_i$ is the number of data points in the $i$-th class, $p$ and $q$ are data points in the $i$-th class

for $(\delta_i, \varepsilon_i)-dp$ guanrantee:

$$\sigma_i \ge \frac{\sqrt{2ln(1.25/\delta_i)} \cdot \triangle_2 f_i }{\varepsilon_i}$$

usually take $\varepsilon=1$ for $\delta$, make sense if $\delta < \frac{1}{n_i}$. acceptable for $\varepsilon < 10$

In [1]:
subset_size = 150

import copy
import torch
import argparse
from models.CLIP import *
from utils.get_data import domainnet
from utils.get_data import get_data
from utils.data_utils import build_subset, split_train_and_val
from utils.server import Server
from utils.clientours import Client
import warnings
import numpy as np
warnings.simplefilter("ignore")

torch.manual_seed(1)
torch.cuda.manual_seed(1) if torch.cuda.is_available() else None

parser = argparse.ArgumentParser(description='DomainFL')
parser.add_argument('-d','--dataset', type=str, default='domainnet', help='Dataset name')
parser.add_argument('-ss','--subset_size', type=int, default=subset_size, help='Subset size')
parser.add_argument('-m','--model', type=str, default='CLIP', help='Model name')
parser.add_argument('-ien','--image_encoder_name', type=str, default='ViT-B-32', help='Image encoder name')
parser.add_argument('-optim','--optimizer', type=str, default='AdamW', help='Optimizer name')
parser.add_argument('-lr','--lr', type=float, default=1e-3, help='Learning rate')
parser.add_argument('-clip','--clip', type=float, default=1, help='Gradient clip')
parser.add_argument('-bs','--batch_size', type=int, default=32, help='Batch size')
parser.add_argument('-le','--local_epochs', type=int, default=1, help='Number of epochs')
parser.add_argument('-warm_up','--warm_up', type=int, default=10, help='Warm up epochs')
parser.add_argument('-gr','--global_rounds', type=int, default=200, help='Number of global rounds')
parser.add_argument('-device','--device', type=str, default='cuda', help='Device')
parser.add_argument('-num_workers','--num_workers', type=int, default=12, help='Number of workers')
parser.add_argument('-eval','--eval_interval', type=int, default=200, help='Log interval')
parser.add_argument('-did','--device_id', type=str, default=0, help='Device ID')
parser.add_argument('-seed','--seed', type=int, default=1, help='Seed')
parser.add_argument('-rw','--regularization_weight', type=float, default=0, help='Regularization weight')
parser.add_argument('-kdw','--kd_loss_weight', type=float, default=0, help='KD loss weight')
parser.add_argument('-sra','--sample_ratio', type=float, default=0.1, help='Sample ratio of all embeddings')
parser.add_argument('-sram','--sample_ratio_method', type=str, default='cluster', help='Sample ratio method (random or cluster, mixed)')
parser.add_argument('-dp','--diff_privacy', type=float, default=0, help='Diff privacy scale')

args, unknown = parser.parse_known_args()

# initialize server
server = Server(args)

# set dataset
dataset = globals()[args.dataset]

# initialize clients
# client image encoder is the same as the global image encoder
clients = []
cls_heads = []
for id, data_name in enumerate(dataset):
    init_image_encoder = copy.deepcopy(server.image_encoder)
    cd = get_data(data_name, server.train_preprocess, server.val_preprocess, args.batch_size, args.num_workers)
    cd = build_subset(cd, args.subset_size)
    cd = split_train_and_val(cd)
    cls_head = server.generate_cls_head(cd, data_name)
    client = Client(args, id, cd.train_dataset, cd.test_dataset, cd.val_dataset, cd.train_loader, cd.test_loader, cd.val_loader, cd.classnames, init_image_encoder, cls_head, data_name)
    clients.append(client)
    cls_heads.append(cls_head)
    del cd

def cal_privacy_preservation_epsilon(distances, ns, sigma=0.05):
    return [distances[i] * np.sqrt(2*np.log(1.25/(1/ns[i]))) / sigma for i in range(len(distances))]

from collections import defaultdict
clients_eps = []
for client in clients:
    train_dataloader = client.train_dataloader
    device = client.device
    protos = defaultdict(list)
    model = client.model
    model.train()

    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        rep = model.base.model.encode_image(inputs)

        for i, yy in enumerate(labels):
            y_c = yy.item()
            protos[y_c].append(rep[i, :].detach().data)

    max_distance_each_client = 0
    max_n = 0
    distances = []
    ns = []
    for i in protos.keys():
        # 求两两torch之差的二范数的最大值
        tensor_list = protos[i]
        max_distance_each_class = 0  # 初始化最大距离
        max_n_each_class = 0
        # 双重循环遍历每一对不同的 tensor 组合
        for j in range(len(tensor_list)):
            for k in range(j + 1, len(tensor_list)):
                # 计算两个 tensor 之间的差的二范数
                distance = torch.norm(tensor_list[j] - tensor_list[k], p=2)
                distance = distance.item() / len(tensor_list)

                # 更新最大距离
                if distance > max_distance_each_class:
                    max_distance_each_class = distance
                    max_n_each_class = len(tensor_list)

        distances.append(max_distance_each_class)
        ns.append(max_n_each_class)

    n_mean = sum(ns) / len(ns)
    clients_eps.append([np.mean(cal_privacy_preservation_epsilon(distances, ns, sigma=s)) for s in [0.01, 0.05, 0.1, 0.5]])

clients_eps = np.array(clients_eps)
clients_eps.mean(axis=0)

/opt/conda/envs/DomainFL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/DomainFL/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


build data Clipart classification head


100%|██████████| 150/150 [00:01<00:00, 106.41it/s]


build data Painting classification head


100%|██████████| 150/150 [00:01<00:00, 120.76it/s]


...

build data Quickdraw classification head


100%|██████████| 150/150 [00:01<00:00, 116.70it/s]


build data Infograph classification head


100%|██████████| 150/150 [00:01<00:00, 109.28it/s]


array([60.73185884, 12.14637177,  6.07318588,  1.21463718])

In [62]:
def cal_privacy_preservation_epsilon(distances, ns, sigma=0.05):
    return [distances[i] * np.sqrt(2*np.log(1.25/(1/ns[i]))) / sigma for i in range(len(distances))]

In [63]:
import defaultdict
clients_eps = []
for client in clients:
    train_dataloader = client.train_dataloader
    device = client.device
    protos = defaultdict(list)
    model = client.model
    model.train()

    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        rep = model.base.model.encode_image(inputs)

        for i, yy in enumerate(labels):
            y_c = yy.item()
            protos[y_c].append(rep[i, :].detach().data)

    max_distance_each_client = 0
    max_n = 0
    distances = []
    ns = []
    for i in protos.keys():
        # 求两两torch之差的二范数的最大值
        tensor_list = protos[i]
        max_distance_each_class = 0  # 初始化最大距离
        max_n_each_class = 0
        # 双重循环遍历每一对不同的 tensor 组合
        for j in range(len(tensor_list)):
            for k in range(j + 1, len(tensor_list)):
                # 计算两个 tensor 之间的差的二范数
                distance = torch.norm(tensor_list[j] - tensor_list[k], p=2)
                distance = distance.item() / len(tensor_list)

                # 更新最大距离
                if distance > max_distance_each_class:
                    max_distance_each_class = distance
                    max_n_each_class = len(tensor_list)

        distances.append(max_distance_each_class)
        ns.append(max_n_each_class)

    n_mean = sum(ns) / len(ns)
    clients_eps.append([np.mean(cal_privacy_preservation_epsilon(distances, ns, sigma=s)) for s in [0.01, 0.05, 0.1, 0.5]])

clients_eps = np.array(clients_eps)
clients_eps.mean(axis=0)

[[78.7242065480313, 15.744841309606262, 7.872420654803131, 1.574484130960626],
 [63.55926843492639, 12.711853686985277, 6.355926843492639, 1.271185368698528],
 [29.472535266615754,
  5.894507053323151,
  2.9472535266615756,
  0.5894507053323151],
 [47.16584317607657, 9.433168635215313, 4.716584317607657, 0.9433168635215314],
 [14.365247953654048,
  2.8730495907308096,
  1.4365247953654048,
  0.28730495907308096],
 [95.28364072673891,
  19.056728145347783,
  9.528364072673892,
  1.9056728145347788]]

In [66]:
clients_eps = np.array(clients_eps)
clients_eps.mean(axis=0)

array([54.76179035, 10.95235807,  5.47617904,  1.09523581])

In [61]:
np.mean(cal_privacy_preservation_epsilon(distances, ns, sigma=0.01))

78.7713252813224

In [58]:
np.mean(cal_privacy_preservation_epsilon(distances, ns, sigma=0.05))

15.75426505626448

In [52]:
np.mean(cal_privacy_preservation_epsilon(distances, ns, sigma=0.1))

7.87713252813224

In [53]:
np.mean(cal_privacy_preservation_epsilon(distances, ns, sigma=0.5))

1.575426505626448

In [20]:
max_distance, max_n = 0, 0
for client in clients:
    train_dataloader = client.train_dataloader
    device = client.device
    protos = defaultdict(list)
    model = client.model
    model.train()

    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        rep = model.base.model.encode_image(inputs)

        for i, yy in enumerate(labels):
            y_c = yy.item()
            protos[y_c].append(rep[i, :].detach().data)

    max_distance_each_client, max_n_each_client = low_bound_privacy_preservation(protos)
    if max_distance_each_client > max_distance:
        max_distance = max_distance_each_client
        max_n = max_n_each_client

print("privacy preservation lower bound:", max_distance * np.sqrt(2*np.log(1.25/(1/max_n))) / 0.05)

Max distance for each client: 0.9616207758585612
Max n: 15
Max distance for each client: 1.5717161723545618
Max n: 7
Max distance for each client: 0.843130953171674
Max n: 17
Max distance for each client: 0.49717400624201846
Max n: 26


KeyboardInterrupt: 

In [ ]:
mean_distance, mean_n = 0, 0
mean_distance_list, mean_n_list = [], []
for client in clients:
    train_dataloader = client.train_dataloader
    device = client.device
    protos = defaultdict(list)
    model = client.model
    model.train()

    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        rep = model.base.model.encode_image(inputs)

        for i, yy in enumerate(labels):
            y_c = yy.item()
            protos[y_c].append(rep[i, :].detach().data)

    mean_distance_each_client, mean_n_each_client = mean_privacy_preservation(protos)
    mean_distance_list.append(mean_distance_each_client)
    mean_n_list.append(mean_n_each_client)

mean_distance = sum(mean_distance_list) / len(mean_distance_list)
mean_n = sum(mean_n_list) / len(mean_n_list)

print("privacy preservation mean:", mean_distance * np.sqrt(2*np.log(1.25/(1/mean_n))) / 0.05)

In [29]:
max_distance * np.sqrt(2*np.log(1.25/(1/max_n))) / 0.5

6.547179796051914

In [27]:
max_distance

1.5717161723545618

In [28]:
np.log(1.25/(1/max_n))

2.169053700369523

In [33]:
mean_distance, mean_n = mean_privacy_preservation(protos)

In [34]:
mean_distance

0.025850887242276535

In [35]:
mean_n

280.0

In [37]:
mean_distance * np.sqrt(2*np.log(1.25/(1/mean_n))) / 0.05

1.7696714831449774

In [3]:
for client in [clients[0]]:
    train_dataloader = client.train_dataloader
    device = client.device
    protos = defaultdict(list)
    model = client.model
    model.train()

    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        rep = model.base.model.encode_image(inputs)

        for i, yy in enumerate(labels):
            y_c = yy.item()
            protos[y_c].append(rep[i, :].detach().data)

In [6]:
max_distance_each_client, max_n_each_client = low_bound_privacy_preservation(protos)

Max distance for each client: 0.9624424616495768
Max n: 15


In [41]:
max_distance_each_client * np.sqrt(2*np.log(1.25/(1/max_n))) / 0.1

23.30301078831292

In [38]:
mean_distance_each_client * np.sqrt(2*np.log(1.25/(1/mean_n))) / 0.1

6.150179973565593

In [32]:
mean_distance_each_client = 0
mean_n = 0
n_list = []
distance_each_client_list = []
for i in protos.keys():
    # 求两两torch之差的二范数的平均值
    tensor_list = protos[i]
    n_list.append(len(tensor_list))
    mean_distance_each_class = 0  # 初始化距离之和
    distance_each_class_list = []  # 初始化距离列表
    # 双重循环遍历每一对不同的 tensor 组合
    for j in range(len(tensor_list)):
        for k in range(j + 1, len(tensor_list)):
            # 计算两个 tensor 之间的差的二范数
            distance = torch.norm(tensor_list[j] - tensor_list[k], p=2)
            distance = distance.item() / len(tensor_list)
            # print(distance)
            distance_each_class_list.append(distance)
    mean_distance_each_class = sum(distance_each_class_list) / len(distance_each_class_list)

    distance_each_client_list.append(mean_distance_each_class)

mean_distance_each_client = sum(distance_each_client_list) / len(distance_each_client_list)
mean_n = sum(n_list) / len(n_list)

In [35]:
mean_distance_each_client

0.20695072323556513

In [36]:
mean_n

66.2

In [11]:
max(distance_each_class_list)

0.5495063708378718

In [14]:
torch.norm(tensor_list[-1] - tensor_list[-2], p=2)

tensor(10.9312, device='cuda:0')

In [16]:
max(distance_each_client_list)

0.6547251347133091

In [60]:
np.min(ns), np.max(ns), np.mean(ns)

(15, 187, 66.2)

In [30]:
np.mean(distances)

0.11528335396011004

In [31]:
np.max(distances)

0.9624424616495768

In [33]:
max_n

15